# Activations

В этой тетрадке мы напишем собственную реализацию функций активации


Запрещено внутри своей реализации создавать класс активации из pytorch и просто применять его. Разрешено исползовать простые ф-ии pytorch типа [torch.exp](https://pytorch.org/docs/stable/generated/torch.exp.html) и т д

Если у ф-ии активации есть дополнительные аргументы, значение по умолчанию должно быть такое же как в реализации PyTorch

**Материалы по pytorch:**

* [PyTorch docs](https://pytorch.org/docs/stable/index.html)


## Prerequirements

```
pip install torch numpy
```

In [9]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

In [10]:
import torch
import torch.nn as nn

## Задание 1
**(0.25 балла)** Напишите свою версию версию функции активации [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#torch.nn.ReLU)




In [11]:
%%write_and_run myrelu.py 
import numpy as np
import torch
import torch.nn as nn

class MyReLU(nn.Module):
    
    def forward(self, input):
        # hint! Входной тензор нужно скопировать,
        # чтобы действиями внутри этого метода не привели к изменению внешнего аргумента
        input_clone = torch.clone(input)
        a = torch.zeros(len(input_clone))
        return torch.maximum(a, input_clone)



## Задание 2
**(0.25 балла)** Напишите свою версию версию функции активации [LeakyReLU](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html#torch.nn.LeakyReLU)




In [12]:
%%write_and_run myleakyrelu.py 

import torch
import torch.nn as nn

class MyLeakyReLU(nn.Module):
    
    def forward(self, input):
        # hint! Входной тензор нужно скопировать,
        # чтобы действиями внутри этого метода не привели к изменению внешнего аргумента
        input_clone = torch.clone(input)
        a = torch.clone(input)
        input_clone[input_clone < 0] = a[a < 0] * 0.01
        return input_clone



## Задание 3
**(0.25 балла)** Напишите свою версию версию функции активации [Sigmoid](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html#torch.nn.Sigmoid)



In [13]:
%%write_and_run mysigmoid.py 

import torch
import torch.nn as nn

class MySigmoid(nn.Module):
    
    def forward(self, input):
        # для этого класса копировать входной тензор уже нет необходимости, почему?
        return 1 / (1 + torch.exp(-input))


## Задание 4
**(0.25 балла)** Напишите свою версию версию функции активации [ELU](https://pytorch.org/docs/stable/generated/torch.nn.ELU.html#torch.nn.ELU)




In [14]:
%%write_and_run myelu.py 

import torch
import torch.nn as nn


class MyELU(nn.Module):        
    def forward(self, input):
        # для этого класса копировать входной тензор уже нет необходимости, почему?
        alpha = 1
        input_clone = torch.clone(input)
        a = torch.clone(input_clone)
        input_clone[input_clone < 0] = alpha * (torch.exp(a[a < 0]) - 1)
        return input_clone


## Тест


In [15]:
import pytest
import torch
import torch.nn as nn

def _check_pytorch_module_was_not_used(file, module):

    file = open(file, mode='r')
    
    assert module not in file.read(), "pytorch module must not be used in you activation implementation"
    
    file.close()

    return

def _test_activation(myactivation, torch_activation):
    print(myactivation)

    with torch.no_grad():

        randinput = torch.rand([100])
        myactivation_output = myactivation(randinput)

        assert id(myactivation_output) != id(randinput), 'pytorch activation function must return new tensor'

        for _ in range(100):
            randinput = torch.rand([5, 5, 5])

            assert torch.allclose(myactivation(randinput), torch_activation(randinput)), 'activation output is not equals to touch ones output'

def test_relu():
    from myrelu import MyReLU

    my_activation = MyReLU()
    
    _check_pytorch_module_was_not_used("myrelu.py", '.ReLU(')
    _test_activation(my_activation, nn.ReLU())

def test_leaky_relu():
    from myleakyrelu import MyLeakyReLU

    my_activation = MyLeakyReLU()
    
    _check_pytorch_module_was_not_used("myleakyrelu.py", '.LeakyReLU(')
    _test_activation(my_activation, nn.LeakyReLU())

def test_sigmoid():
    from mysigmoid import MySigmoid

    my_activation = MySigmoid()
    
    _check_pytorch_module_was_not_used("mysigmoid.py", '.MySigmoid(')
    _test_activation(my_activation, nn.Sigmoid())

def test_elu():
    from myelu import MyELU

    my_activation = MyELU()
    
    _check_pytorch_module_was_not_used("myelu.py", '.MyELU(')
    _test_activation(my_activation, nn.ELU())


test_relu()
test_leaky_relu()
test_sigmoid()
test_elu()

MyReLU()
MyLeakyReLU()
MySigmoid()
MyELU()
